# Introduction: Interferometry vs. photometry


Let's say we have a spherical star whose intensity map, projected into a 2D at a specific vieweing orientation, is defined as $I(x,y)$. Finding the total brightness of the star at that orientation is a sum over the visible surface of the star: 

 $$ F = \iint I(x,y)dx dy $$
 
 
 This problem of fiding the disk-integrated brightness of a star--the observable quantity of photometry--has been solved in the [starry package](https://arxiv.org/abs/1810.06559) in a rather elegant way that, among other things, permits a description of the information content of photometric data. This is done first by representing the surface map of a star using [spherical harmonics](https://en.wikipedia.org/wiki/Spherical_harmonics). If the 3d surface of a star is represented using spherical harmonic coefficients $\mathbf{y}$, then we can write the intensity map as:
 
$$I(x,y) = \mathbf{\tilde{y}}^\top(x,y) \ \mathbf{R} \ \mathbf{y}$$

where $\mathbf{\tilde{y}}^\top(x,y)$ is the spherical harmonic basis, $\mathbf{R}$ is the rotation matrix into the correct viewing orientation with the viewer at $+\infty$ along the z axis and $\mathbf{y}$ is the vector of spherical harmonic coefficients. Because spherical harmonics form an orthonormal basis on a unit sphere, *any* map can be represented using a sufficiently high order expansion in the spherical harmonics. This makes it a natural choice to represent the surface of a star.

The real achievement of starry was to find an analytic way of performing the surface integral where a star is represented using spherical harmonics. This makes it extremely fast to compute photometric observables--light curves-- as a star rotates or even as a planet occults it. 

In this notebook, I will attempt to show that it is possible to use the same elegant description of the surface of a star in terms of spherical harmonics to find analytic observables used in interferometry. Recall that interferometric observations record a quantity called the visibility. The van-Cittert Zernike theorem relates the intensity map of a star to its visibility using a different kind of double integral from the one for photometry--the Fourier transform:

$$ V(u,v) = \iint I(x,y) e^{i(ux + vy)} dxdy $$

Where $V$ is the visibility at baseline (u,v). This integral is similar but not identical to the one from photometry, and a similar intuition should be able to help solve it. First, we must find a way to take the Fourier transform of the surface map expressed as an expansion in spherical harmonics. 

$$ V(u,v) = \iint \mathbf{\tilde{y}}^\top(x,y) e^{i(ux + vy)} dxdy \ \ \mathbf{R} \ \mathbf{y}$$ 

where we can pull out $\mathbf{R} \ \mathbf{y}$ from the integral as they do not depend on x and y. Now, we must find the Fourier transform of the spherical harmonic basis. Although this might appear difficult, I have found that half the terms in $\mathbf{\tilde{y}}^\top(x,y)$ (those with l+m even) map perfectly onto a polynomial basis defined on a unit disk which has an analytic Fourier transform--the Zernikes. The other half have an analytic Fourier transform in terms of spherical Bessel functions. Let's split the spherical harmonic basis into two complementary bases, called the [hemispheric harmonics (HSH) and the complementary hemispheric harmonics (CHSH)](https://opg.optica.org/oe/fulltext.cfm?uri=oe-27-26-37180&id=423949):
$$ \mathbf{\tilde{y}} = \mathbf{\tilde{y}}_{\mathrm{HSH}} + \mathbf{\tilde{y}}_{\mathrm{CHSH}}$$
What we must do is find another change of basis matrix $\mathbf{A}$ for which

$$ \mathbf{{z}} = \mathbf{A} \ \mathbf{y}_{\mathrm{HSH}} $$

where $\mathbf{\tilde{z}}$ is the zernike basis. If we can find such a matrix $\mathbf{A}$, we can solve the Fourier integral as follows:

$$ V(u,v) =  \left((\mathbf{A}^{-1}\widehat{\mathbf{\tilde{z}}})^\top + \widehat{\mathbf{\tilde{y}}}_{\mathrm{CHSH}}^\top \right)\mathbf{R}\mathbf{y}$$

where $\widehat{\mathbf{\tilde{z}}}$ is the Zernike solution vector containing Fourier transforms of each term in the Zernike basis and $\widehat{\mathbf{\tilde{y}}}_{\mathrm{CHSH}}$ is the CHSH solution vector containing Fourier transforms of each CHSH term. In practice we do not do this all in the Cartesian basis, but use $V(\rho, \phi)$ instead of $V(u, v)$ and $r, \theta$ instead of $x, y$, allowing us to express the Zernike and spherical harmonic bases as separable products of Zernike polynomials and Legendre functions times cos $m \phi$ or sin $m \phi$ respectively. 

In [312]:
import sympy as sp
from sympy import symbols, sin, cos, Matrix, Eq, Rational, floor, sqrt
from sympy import simplify, factorial, pi, binomial, factor, expand, collect, gamma
from sympy.functions.special.tensor_functions import KroneckerDelta
from sympy import init_printing
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
import pandas as pd
from sympy import latex
from scipy.optimize import curve_fit
from IPython.display import display, Math
from hswfs import zernike

Lets show terms in the zernike basis $\mathbf{\tilde{z}}(\rho,\phi)$ up to N of 5:

In [313]:
r, theta = sp.symbols('r, theta')

def zpoly(j, r):
    """ returns the jth term of the zernike polynomial basis"""
    n = int(np.ceil((-3 + np.sqrt(9 + 8 * j)) / 2))
    m = int(2 * j - n * (n + 2))
    res = 0
    for s in range(0,int((n-np.abs(m))/2)+1):
        res += Rational((-1)**s * factorial(n-s) / (factorial(s) * factorial((n+np.abs(m))/2 -s) * factorial((n-np.abs(m))/2 - s))) * r**(n-2*s)
    if m<0:
        return res*0
    elif m>0:
        return res
    else:
        return res
    
jmax = lambda nmax: (nmax**2+3*nmax)//2
zbasis = Matrix([zpoly(j, r) for j in range(jmax(5)+1)])
zbasis


Matrix([
[                      1],
[                      0],
[                      r],
[                      0],
[             2*r**2 - 1],
[                   r**2],
[                      0],
[                      0],
[           3*r**3 - 2*r],
[                   r**3],
[                      0],
[                      0],
[    6*r**4 - 6*r**2 + 1],
[        4*r**4 - 3*r**2],
[                   r**4],
[                      0],
[                      0],
[                      0],
[10*r**5 - 12*r**3 + 3*r],
[        5*r**5 - 4*r**3],
[                   r**5]])

Now, let's show the spherical harmonic basis up to lmax of 5:

In [314]:
def hsh(n, r):
    l = Rational(floor(sqrt(n)))
    m = Rational(n - floor(sqrt(n))**2 - floor(sqrt(n)))
    if m<0:
        angular = 0
    elif m>0:
        angular = 1
    else:
        angular = 1
    m = np.abs(m)
    if (l-m)%2 != 0:
        return 0
        print("CHSH")
    else:
        res = 0
        for k in range(0,l-m+1):
            for j in range(0,int(k/2)+1):
                res += 2**l * (gamma(Rational(l+m+k-1, 2)+1)/(factorial(k) * factorial(l-m-k) * gamma(Rational(-l+m+k-1, 2)+1))) * binomial(Rational(k,2),j) * (-1)**j * r**(2*j+m)
        return res*angular

nmax = lambda lmax: lmax**2 + 2*lmax
hshbasis = Matrix([hsh(n, r) for n in range(nmax(5)+1)])
hshbasis
    

Matrix([
[                             1],
[                             0],
[                             0],
[                             r],
[                             0],
[                             0],
[                  1 - 3*r**2/2],
[                             0],
[                        3*r**2],
[                             0],
[                             0],
[                             0],
[                             0],
[              -15*r**3/2 + 6*r],
[                             0],
[                       15*r**3],
[                             0],
[                             0],
[                             0],
[                             0],
[        35*r**4/8 - 5*r**2 + 1],
[                             0],
[         -105*r**4/2 + 45*r**2],
[                             0],
[                      105*r**4],
[                             0],
[                             0],
[                             0],
[                             0],
[    

## Change of basis matrix:

Now lets write a single spherical harmonic in the zernike basis. Writing these vectors as the columns of a matrix for each term in the zernike basis should give us our change of basis matrix (or is it the inverse??):

In [315]:
basis = Matrix([zpoly(j, r) for j in range(jmax(6)+1)]).T
def Coefficient(expression, term):
    """Return the coefficient multiplying `term` in `expression`."""
    # Get the coefficient
    coeff = expression.coeff(term)
    if term==1:
        coeff = expression.subs(r, 0)
    # Set any non-constants in this coefficient to zero. If the coefficient
    # is not a constant, this is not the term we are interested in!
    coeff = coeff.subs(r, 0)
    return coeff

zernike_coeffs = Matrix([Coefficient(hsh(42, r),term) for term in basis])
zernike_coeffs

Matrix([
[      1],
[      0],
[      0],
[      0],
[      0],
[  -21/2],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[  189/8],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[-231/16]])

In [316]:
display(Math(latex(zernike_coeffs.T*basis.T))) #basis.T because basis was already defined with a transpose above
display(Math(latex(hsh(42, r))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [317]:
def A(lmax):
    jmax = (lmax**2+3*lmax)//2 #the array size of the zernike basis we need
    nmax = lmax**2 + 2*lmax #the array size of the spherical harmonic basis we need
    res = sp.zeros(nmax+1,jmax+1)
    basis = Matrix([zpoly(j, r) for j in range(jmax+1)]).T
    counter=0
    for n in range(nmax+1):
        hsh_expr = hsh(n, r)
        if hsh_expr==0:
            for index, term in enumerate(basis):         
                res[counter] = 0
                counter +=1
        else:
            for index, term in enumerate(basis):
                res[counter] = Coefficient(hsh(n, r),term)
                counter+=1
    return res

### Here's our change of basis matrix A!

In [318]:
lmax = 4
zernike_basis = Matrix([zpoly(j, r) for j in range(jmax(lmax)+1)]).T
poly = Matrix([0 for j in range(nmax(lmax)+1)]).T
poly[13]=1
chbasis = A(lmax)
soln = chbasis.T*poly.T
display(Math(latex(chbasis.T)+latex(poly.T)+"="+latex(soln)))

<IPython.core.display.Math object>